### files2rouge

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Jun  3 05:29:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy as np
import pandas as pd
import torch

In [4]:
# setting for pyrouge - used below
!git clone https://github.com/bheinzerling/pyrouge.git

%cd pyrouge
!apt install libxml-parser-perl
!pip install -e .
!git clone https://github.com/andersjo/pyrouge.git rouge

%cd rouge/tools/ROUGE-1.5.5/data
!rm WordNet-2.0.exc.db
!perl ./WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db
!pyrouge_set_rouge_path /content/drive/MyDrive/MDS/GraphSum/pyrouge/rouge/tools/ROUGE-1.5.5
!chmod 777 /content/drive/MyDrive/MDS/GraphSum/pyrouge/rouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl
!python -m pyrouge.test
!pip install -U git+https://github.com/pltrdy/pyrouge

Cloning into 'pyrouge'...
remote: Enumerating objects: 551, done.
remote: Total 551 (delta 0), reused 0 (delta 0), pack-reused 551
Receiving objects: 100% (551/551), 123.17 KiB | 7.25 MiB/s, done.
Resolving deltas: 100% (198/198), done.
/content/pyrouge
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libauthen-sasl-perl libdata-dump-perl libencode-locale-perl
  libfile-listing-perl libfont-afm-perl libhtml-form-perl libhtml-format-perl
  libhtml-parser-perl libhtml-tagset-perl libhtml-tree-perl
  libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libio-html-perl
  libio-socket-ssl-perl liblwp-mediatypes-perl liblwp-protocol-https-perl
  libmailtools-perl libnet-http-perl libnet-smtp-ssl-perl libnet-

In [ ]:
!git clone https://github.com/pltrdy/files2rouge.git     
%cd files2rouge
!python setup_rouge.py
!python setup.py install

In [6]:
%%capture
!pip install bert-score
!pip install rouge
!pip install summ-eval

#### Calculate ROGUE and BS without adding updated info

In [ ]:
# !files2rouge /content/drive/MyDrive/MDS/wcep_custom/baseline_1/aftold_model_txt.src /content/drive/MyDrive/MDS/wcep_custom/baseline_1/test.txt.tgt -e ". "

Can't load ROUGE settings in '/usr/local/lib/python3.7/dist-packages/files2rouge-2.1.0-py3.7.egg/files2rouge/settings.json'. Check that the file exists or initialize it with 'setup_rouge.py'


In [ ]:
%ls /content/drive/MyDrive/MDS/wcep_custom/bs1_train/filtered_non/without_update_src

bef.test.txt.src  output_intermediate.txt  test.txt.tgt


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered/without_update_src
from rouge import FilesRouge

files_rouge = FilesRouge()
# hyp_path = '../gen_summ/gen_summ.test.tgt'
hyp_path = '../aft_test_rouge_txt.src'
# hyp_path = 'bef.test.txt.src'
ref_path = '../test.txt.tgt'
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
scores

In [ ]:
%cd /content
!git clone https://github.com/Yale-LILY/SummEval
!pip install -U  git+https://github.com/bheinzerling/pyrouge.git
!export ROUGE_HOME=/usr/local/lib/python3.7/dist-packages/summ_eval/ROUGE-1.5.5/ 

  Cloning https://github.com/bheinzerling/pyrouge.git to /tmp/pip-req-build-jme5imbr
  Running command git clone -q https://github.com/bheinzerling/pyrouge.git /tmp/pip-req-build-jme5imbr
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191924 sha256=3ec1bf84e3f52b24c748ab6613c6515791bdb0a29fc0558cd35d79bf420c36f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-mum7k9vw/wheels/2f/1c/55/4e264b4e45f2a6f39a50e95cff45e57b54f202db830f125bb2
Successfully built pyrouge
  Attempting uninstall: pyrouge
    Found existing installation: pyrouge 0.1.3
    Uninstalling pyrouge-0.1.3:
      Successfully uninstalled pyrouge-0.1.3


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered/without_update_src
!calc-scores --config-file=/content/SummEval/evaluation/summ_eval/examples/basic.config --metrics "rouge" --summ-file '../aft_test_rouge_txt.src' --ref-file '../test.txt.tgt' --output-file rouge.jsonl --eos " . " --aggregate True

/content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered/without_update_src
Please run the following command and add it to your startup script: 
 export ROUGE_HOME=/usr/local/lib/python3.7/dist-packages/summ_eval/ROUGE-1.5.5/
Please also run this command: 
 pip install -U  git+https://github.com/bheinzerling/pyrouge.git


In [ ]:
check_ee = []
with open('test.txt.tgt') as fread:
    for line in fread.readlines():
        check_ee.append(line.strip())
print(len(check_ee))

177


### With low rouge, calculate BS on added summaries

In [ ]:
# with open("gen_summ/gen_summ.test.tgt") as f:
with open("aft_model_sec_txt.src") as f:
    cands = [line.strip() for line in f]

with open("test.txt.tgt") as f:
    refs = [line.strip() for line in f]
assert len(cands)==len(refs)

In [ ]:
from bert_score import score

P, R, F1 = score(cands, refs, lang="en", verbose=True)
print(f"System level F1 score: {F1.mean():.3f}")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 492.70 seconds, 0.36 sentences/sec
System level F1 score: 0.889


### Check if the shape of rouge dataframe is equal to generated summaries

In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/baseline_1/gen_summ

/content/drive/MyDrive/MDS/wcep_custom/baseline_1/gen_summ


In [ ]:
rouge_result_df = pd.read_csv('rouge_result.csv')
check_genlen = []
with open('gen_summ.test.tgt', 'r') as fread:
    for line in fread.readlines():
        check_genlen.append(line.strip())
print(len(check_genlen))
print(bool(len(rouge_result_df)==len(check_genlen)))

371
True


#### Filtering

- Filter the summaries to solve the problems that generated summaries might appear in the documents

In [ ]:
# check summ and rouge
%cd /content/drive/MyDrive/MDS/wcep_custom/baseline_2/gen_summ
rouge_df = pd.read_csv('rouge_result.csv')
rouge_scores = list(rouge_df['rouge-L-f'].values)
less_idx = [idx for idx in range(len(rouge_scores)) if rouge_scores[idx]<=0.4]
eq_idx = [idx for idx in range(len(rouge_scores)) if rouge_scores[idx]==1.0]
# gre_idx = [idx for idx in range(len(rouge_scores)) if rouge_scores[idx]>np.mean(rouge_scores) and rouge_scores[idx]!=1.0]

/content/drive/MyDrive/MDS/wcep_custom/baseline_2/gen_summ


In [ ]:
print(f'ROUGE less than 0.4: {len(less_idx)}')
print(f'ROGUE equal to 1: {len(eq_idx)}')

ROUGE less than 0.4: 170
ROGUE equal to 1: 40


In [ ]:
less_idx[:10]

[0, 2, 3, 6, 7, 8, 10, 11, 17, 18]

In [ ]:
gen_summ = []
tgt_summ = []
src_docs = []
with open('gen_summ.test.tgt', 'r') as fread:
    for line in fread.readlines():
        gen_summ.append(line.strip())

with open('../test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        tgt_summ.append(line.strip())
with open('../aft_test_model_txt.src') as fread:
    for line in fread.readlines():
        src_docs.append(line.strip())

In [ ]:
len(gen_summ)==len(tgt_summ)==len(src_docs)

True

In [ ]:
src_docs[11]

'The 2016 NRL Grand Final was a rugby league match between the Melbourne Storm and the Cronulla-Sutherland Sharks  to determine the premiers of the National Rugby League for the 2016 season. The match was held at Sydneys ANZ Stadium on Sunday 2 October. Melbournes last premiership was in 2012 while Cronulla had never won a premiership in the clubs 49 year history.\\c\\cThe Sharks won the match with a 14–12 victory before 83,625 spectators, earning the club its first premiership title.  Cronulla forward Luke Lewis was awarded the Clive Churchill Medal as the best player on ground\\c\\cThe Melbourne Storm and the Cronulla-Sutherland Sharks met twice during the regular NRL season; in round 4 and round 26. In round 4, the Sharks defeated the Storm 14-6 at Southern Cross Group Stadium in Cronulla, New South Wales to hand the Storm their first loss of the season; it was also the start of a sixteen-match unbeaten streak for the Sharks, while in round 26, the Storm defeated the Sharks 26-6 in 

In [ ]:
tgt_summ[11]

'{"The 2016 NRL Grand Final was a rugby league match between the Melbourne Storm and the Cronulla-Sutherland Sharks  to determine the premiers of the National Rugby League for the 2016 season. The match was held at Sydney\'s ANZ Stadium on Sunday 2 October. Melbourne\'s last premiership was in 2012 while Cronulla had never won a premiership in the club\'s 49 year history.'

In [ ]:
gen_summ[11]

'In Australian rugby league, the Cronulla-Sutherland Sharks defeat the Melbourne Storm 14–12 to win their first premiership after 50 years in the competition.'

In [ ]:
type(src_docs[0])

str

In [ ]:
%ls /content/drive/MyDrive/MDS/wcep_custom/bs2_test/filtered/without_update_src

bef.test.txt.src  test.txt.tgt


In [8]:
# Calculate BERTScore
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered_non
from bert_score import score
with open("gen_summ_tuned/gen_summ_tuned.test.tgt") as f:
# with open("aft_model_txt.src") as f:
# with open('without_update_src/bef.test.txt.src') as f:
    cands = [line.strip() for line in f]

with open("test.txt.tgt") as f:
    refs = [line.strip() for line in f]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
print(f"System level F1 score: {F1.mean():.3f}")

/content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered_non


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 17.03 seconds, 11.81 sentences/sec
System level F1 score: 0.966


In [ ]:
%ls /content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered_non/gen_summ

gen_summ.test.tgt  rouge_result.csv


In [ ]:
check_csv = pd.read_csv('/content/drive/MyDrive/MDS/wcep_custom/bs3_valid/filtered/gen_summ/rouge_result.csv')
print(check_csv.shape)

calculate rouge to find more different ones

In [14]:
# exp.1
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered_non
orig_tgt = []
gen_summ = []
# with open('aft_test_model_txt.src', 'r') as fread:
#     for line in fread.readlines():
#         orig_src.append(line.strip())
with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        orig_tgt.append(line.strip())
# with open('gen_summ/gen_summ.test.tgt', 'r') as fread:
with open('gen_summ_tuned/gen_summ_tuned.test.tgt', 'r') as fread:
    for line in fread.readlines():
        gen_summ.append(line.strip())
# rouge_orig_record = []
# for idx in range(len(orig_tgt)):
#     src = orig_tgt[idx]
#     tgt = orig_tgt[idx]
    # scores = rouge.get_scores(src, tgt)
    # rouge_src_tgt.append(scores[0]['rouge-l']['f'])
    # rouge_src_tgt.append(scores[0]['rouge-l']['f'])
# rouge_result = pd.read_csv('gen_summ/rouge_result.csv')

/content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered_non


In [15]:
cal_token_gen = []
cal_token_tgt = []
for idx in range(len(orig_tgt)):
    hyp = gen_summ[idx]
    ref = orig_tgt[idx].lstrip('{"')
    cal_token_gen.append(len(hyp.split()))
    cal_token_tgt.append(len(ref.split()))

print(np.min(cal_token_gen))
print(np.max(cal_token_gen))
print(np.mean(cal_token_gen))
print(np.median(cal_token_gen))

print(np.min(cal_token_tgt))
print(np.max(cal_token_tgt))
print(np.mean(cal_token_tgt))
print(np.median(cal_token_tgt))

33
858
610.7860696517413
731.0
29
38913
3905.945273631841
2081.0


#### low rouge: calculate `BLEU`, `BS`, `METEOR`

In [7]:
%ls /content/drive/MyDrive/MDS/wcep_custom/bs1_test/filtered_non/without_update_src

bef.test.txt.src  output_intermediate.txt


In [9]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered_non
gen_summ = []
tgt = []
with open('gen_summ_tuned/gen_summ_tuned.test.tgt', 'r') as fread:
# with open('without_update_src/bef.test.txt.src') as fread:
# with open('aft_model_txt.src', 'r') as fread:
    for line in fread.readlines():
        gen_summ.append(line.strip())
with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        tgt.append(line.strip())
assert len(gen_summ)==len(tgt)

/content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered_non


In [18]:
len(tgt)

201

In [10]:
%%capture
!pip install nltk -U

In [11]:
# extract the gen_summ and tgt
gen_summ_sel = [gen_summ[idx].lstrip('{"').lstrip("{'").rstrip() for idx in range(len(gen_summ))]
tgt_sel = [tgt[idx].lstrip('{"').lstrip("{'").rstrip() for idx in range(len(tgt))]
assert len(gen_summ_sel)==len(tgt_sel)

In [12]:
# calculate BLEU
from nltk.translate.bleu_score import sentence_bleu
tokens_sentences = []
candidates = []
scores_1gram, scores_2gram, scores_3gram, scores_4gram = [], [], [], []
for idx in range(len(tgt_sel)):
    ref_sentences = tgt_sel[idx].split('. ')
    tokens_sentence = []
    for sentence in ref_sentences:
        tokens_sentence.append(sentence.split())
    tokens_sentences.append(tokens_sentence)
for idx in range(len(gen_summ_sel)):
    candidate = gen_summ_sel[idx].split()
    candidates.append(candidate)
for idx in range(len(gen_summ_sel)):
    scores_1gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(1, 0, 0, 0)))
    scores_2gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(0, 1, 0, 0)))
    scores_3gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(0, 0, 1, 0)))
    scores_4gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(0, 0, 0, 1)))
print(np.mean(scores_1gram))
print(np.mean(scores_2gram))
print(np.mean(scores_3gram))
print(np.mean(scores_4gram))

0.5998661290461802
0.7784555190020281
0.7898688996513129
0.7641391440730041


In [13]:
# calculate METEOR
import nltk
from nltk.translate.meteor_score import meteor_score as mt
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
scores = []
for idx in range(len(tgt_sel)):
    ref_sentences = tgt_sel[idx].split('. ')
    ref_tokenized = [word_tokenize(sent) for sent in ref_sentences]
    can = word_tokenize(gen_summ_sel[idx])
    scores.append(mt(ref_tokenized, can))
print(np.mean(scores))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


0.49423801291128944


#### high rouge: filter the duplciated summaries, than calculate `BLEU`, `BS`, `METEOR` with old-title-pages and updated info respectively

In [ ]:
wceptest = torch.load('../wceptest.pt')

In [ ]:
wceptest[0]['summary']

True

In [ ]:
dup_ids = []
for idx in range(len(gen_summ)):
    if gen_summ[idx] in wceptest[idx]['summary']:
        dup_ids.append(idx)

In [ ]:
dup_hypsumm = [gen_summ[idx] for idx in dup_ids]
dup_refsumm = [wceptest[idx]['summary'] for idx in dup_ids]

In [ ]:
more_idx = []
for idx in range(len(dup_hypsumm)):
    if len(word_tokenize(dup_hypsumm[idx])) > len(word_tokenize(dup_refsumm[idx])):
        more_sents.append()
        more_idx.append(idx)


61